In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from image_captioner import ImageEncoder
from coco_loader import get_coco_loader
from paths import paths
import yaml
import torch
from image_transforms import image_transform_index

In [3]:
with open(paths["config"]) as f:
    config = yaml.safe_load(f)

IMAGE_SIZE = config["image_size"]
PATCH_SIZE = config["patch_size"]
NUM_WORKERS = config["num_workers"]
BATCH_SIZE = config["batch_size"]
transformer_encoder_config = config["transformer_encoder"]

In [4]:
# Set device.
if "device" in config:
    device = config["device"]
else:
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps" if torch.backends.mps.is_available() else "cpu"
    )
print(f"You are using {device}.")

You are using cuda.


In [ ]:
model = ImageEncoder(IMAGE_SIZE, PATCH_SIZE, transformer_encoder_config).to(device)
train_batches = iter(
    get_coco_loader(
        "train", BATCH_SIZE, 100, image_transform_index["train"], 0, NUM_WORKERS
    )
)

In [ ]:
image, caption = next(train_batches)
image = image.to(device)
positions = torch.randint(0, 196, (49,), device=image.device)

In [7]:
positions

tensor([121,   0, 158, 102, 126, 177, 176, 144, 135, 185, 137, 121, 185,  64,
        153,   6, 167,  85,  55, 170,  60,  55, 103,  73, 193, 182, 152,  55,
         87,  17, 178,  64,  91,  66,  20, 103, 185,  21, 143,  75, 185,  70,
         54,  30,  26,  96,  95,  94, 169], device='cuda:0')

In [8]:
image.dtype

torch.float32

In [9]:
random_patches = model(image, positions=positions)

In [10]:
random_patches.shape

torch.Size([128, 49, 512])